In [1]:
import pandas as pd
import numpy as np
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
import folium
import json
import pydeck as pdk
import geopandas as gpd

In [2]:
yp_df = pd.read_excel("../../data/moving_population/ingu_hour_yeungdeungpo.xlsx",sheet_name=0)

In [3]:
mp_df = pd.read_excel("../../data/moving_population/ingu_hour_mapo.xlsx",sheet_name=0)

In [5]:
ingu_var = [x for x in yp_df.columns if '_ingu' in x and len(x)>13]
print(len(ingu_var))
yp_df_2 = pd.melt(yp_df,['hour','latitude','longitude','gu_name','dong_name'],value_vars=ingu_var,var_name='성별연령대',value_name='인구')
yp_df_2.head()
yp_df_2['연령대'] = yp_df_2['성별연령대'].str.slice(stop=7)
yp_df_2['성별'] = yp_df_2['성별연령대'].str.slice(start=8)
yp_df_2 = yp_df_2.drop("성별연령대", axis=1)
yp_df_2['성별'] = yp_df_2.apply(lambda row:row['성별'].split('_')[0],axis=1)
yp_df_2.head()

18


,hour,latitude,longitude,gu_name,dong_name,인구,연령대,성별
0,0,37.510243,126.904651,영등포구,영등포본동,0.185763,under10,male
1,0,37.516164,126.917051,영등포구,영등포본동,0.112386,under10,male
2,0,37.512080,126.911425,영등포구,영등포본동,0.000000,under10,male
3,0,37.511581,126.901812,영등포구,영등포본동,0.417325,under10,male
4,0,37.514795,126.913667,영등포구,영등포본동,0.076697,under10,male


In [6]:
yp_df_2.groupby('dong_name')['latitude'].describe()

,count,mean,std,min,25%,50%,75%,max
dong_name,,,,,,,,
당산1동,128736.0,37.523915,0.002287,37.519236,37.521928,37.523744,37.525955,37.529119
당산2동,265320.0,37.533928,0.004710,37.526030,37.530034,37.533226,37.537741,37.544031
대림1동,86832.0,37.493676,0.002038,37.489070,37.492214,37.493588,37.495363,37.497191
대림2동,99360.0,37.489601,0.002182,37.485439,37.487713,37.489491,37.491287,37.494447
대림3동,160704.0,37.500029,0.003692,37.492621,37.497139,37.499851,37.502557,37.508383
도림동,101088.0,37.509134,0.001623,37.504353,37.507972,37.509314,37.510257,37.512071
문래동,310140.0,37.516519,0.002813,37.508830,37.514282,37.516532,37.518739,37.522736
신길1동,123552.0,37.512423,0.003080,37.505796,37.509868,37.512128,37.515268,37.518422
신길3동,91152.0,37.506210,0.001609,37.503009,37.504846,37.506173,37.507539,37.510271


In [7]:
mp_df_2 = pd.melt(mp_df,['hour','latitude','longitude','gu_name','dong_name'],value_vars=ingu_var,var_name='성별연령대',value_name='인구')
mp_df_2.head()
mp_df_2['연령대'] = mp_df_2['성별연령대'].str.slice(stop=7)
mp_df_2['성별'] = mp_df_2['성별연령대'].str.slice(start=8)
mp_df_2 = mp_df_2.drop("성별연령대", axis=1)
mp_df_2.head()
mp_df_2['성별'] = mp_df_2.apply(lambda row:row['성별'].split('_')[0],axis=1)
mp_df_2.head()

,hour,latitude,longitude,gu_name,dong_name,인구,연령대,성별
0,0,37.548776,126.951326,마포구,아현동,0.365343,under10,male
1,0,37.553301,126.955255,마포구,아현동,0.245545,under10,male
2,0,37.553755,126.955818,마포구,아현동,0.575684,under10,male
3,0,37.546512,126.949078,마포구,아현동,0.017785,under10,male
4,0,37.550584,126.952444,마포구,아현동,0.580266,under10,male


In [8]:
from branca.colormap import linear

colormap = linear.YlGn_09.scale(
    yp_df_2['인구'].min(),
    yp_df_2['인구'].max())
colormap

## 각 동별(row), 시간별(col), 성별(남/녀/전체) 인구 테이블

In [103]:
yp_groupby = yp_df_2[['hour','dong_name','성별','인구']].groupby(['hour','dong_name','성별'],as_index=False).sum()
mp_groupby = mp_df_2[['hour','dong_name','성별','인구']].groupby(['hour','dong_name','성별'],as_index=False).sum()
ypmp_groupby = pd.concat([yp_groupby,mp_groupby])

## 유동 인구 테이블로 변환 (새벽3시 기준)

### 거주 인구 데이터 불러오기

In [102]:
enrolled_ingu = pd.read_csv('서울시 주민등록인구 (동별) 통계.txt', sep="\t", header=2)
target_enrolled_ingu = enrolled_ingu[(enrolled_ingu['자치구']=='마포구') | (enrolled_ingu['자치구']=='영등포구')]
target_enrolled_ingu = target_enrolled_ingu[['행정동','계','남자','여자']]
target_enrolled_ingu.head()

,행정동,계,남자,여자
220,소계,"385,925","181,303","204,622"
221,용강동,"22,441","10,655","11,786"
222,대흥동,"13,575","6,196","7,379"
223,염리동,"13,486","6,338","7,148"
224,신수동,"23,877","11,605","12,272"


In [200]:
move_ypmp_groupby = ypmp_groupby.copy()
move_ypmp_groupby['index'] = range(move_ypmp_groupby.shape[0])
move_ypmp_groupby = move_ypmp_groupby.set_index('index')
# move_ypmp_groupby
diff_list = []
for _hour in range(1):
    diff = np.array(move_ypmp_groupby[move_ypmp_groupby['hour']==_hour]['인구']) - np.array(move_ypmp_groupby[move_ypmp_groupby['hour']==3]['인구'])
    length = len(diff)
    move_ypmp_groupby.iloc[_hour*length:(_hour+1)*length,3] = diff
move_ypmp_groupby
move_ypmp_groupby.to_csv()

,hour,dong_name,성별,인구
index,,,,
0,0,당산1동,female,114.540847
1,0,당산1동,male,289.717264
2,0,당산2동,female,203.303645
3,0,당산2동,male,462.077516
4,0,대림1동,female,-7.363765
...,...,...,...,...
1627,23,염리동,male,2061.987999
1628,23,용강동,female,4466.298791
1629,23,용강동,male,4513.142324


## 여의도, 마포구 데이터 이어주기 (concat)

In [58]:
total_groupby = pd.concat([yp_groupby, mp_groupby])
total_groupby

,hour,dong_name,성별,인구
0,0,당산1동,female,4960.996613
1,0,당산1동,male,5268.878628
2,0,당산2동,female,6869.698127
3,0,당산2동,male,6887.241337
4,0,대림1동,female,3485.915988
...,...,...,...,...
763,23,염리동,male,2061.987999
764,23,용강동,female,4466.298791
765,23,용강동,male,4513.142324
766,23,합정동,female,4827.601396


In [ ]:
  
move_groupby_dong = []
for x in groupby_dong:
    temp_df = x.copy()
    temp_df['인구'] = x['인구']-groupby_dong[3]['인구']
    move_groupby_dong.append(temp_df)

move_groupby_dong[0].describe()
# print(len(move_groupby_dong))

In [38]:
groupby_dong = []
for _hour in range(24):
    temp_df = yp_df_2[yp_df_2['hour']==_hour].groupby('dong_name').sum()['인구'].to_frame()
    temp_df['dong'] = temp_df.index
    temp_df['hour'] = _hour
    temp_df['index'] = range(temp_df.shape[0])
    temp_df = temp_df.set_index('index')
    groupby_dong.append(temp_df)
    
move_groupby_dong = []
for x in groupby_dong:
    temp_df = x.copy()
    temp_df['인구'] = x['인구']-groupby_dong[3]['인구']
    move_groupby_dong.append(temp_df)

move_groupby_dong[0].describe()
# print(len(move_groupby_dong))

,인구,hour
count,18.000000,18.0
mean,275.214676,0.0
std,488.972150,0.0
min,-51.322060,0.0
25%,18.907939,0.0
50%,85.893435,0.0
75%,276.060767,0.0
max,1930.018625,0.0


## 각 동별(row), 시간별(col), 성별(남/녀/전체) 인구 테이블

## 각 동별(row), 특정 시간의 남/녀/전체(col) 인구 테이블

## 각 동별(row), 특정 시간대의 남/녀/전체(col) 인구 테이블



In [39]:
move_groupby_dong[0]

,인구,dong_name,hour
index,,,
0,404.258111,당산1동,0
1,665.381161,당산2동,0
2,-51.322060,대림1동,0
3,65.283159,대림2동,0
4,150.597540,대림3동,0
5,-12.517122,도림동,0
6,121.662073,문래동,0
7,74.895458,신길1동,0
8,-6.453585,신길3동,0


In [ ]:
groupby_dong = []
for _hour in range(24):
    temp_df = mp_df_2[mp_df_2['hour']==_hour].groupby('dong_name').sum()['인구'].to_frame()
    temp_df['dong_name'] = temp_df.index
    temp_df['hour'] = _hour
    temp_df['index'] = range(temp_df.shape[0])
    temp_df = temp_df.set_index('index')
    groupby_dong.append(temp_df)
    
for x in groupby_dong:
    temp_df = x.copy()
    temp_df['인구'] = x['인구']-groupby_dong[3]['인구']
    move_groupby_dong.append(temp_df)

In [18]:
geo_path = 'geodata/seoul_yp_mp_map.geojson'
geo_str = json.load(open(geo_path, encoding='utf-8'))
# geo_str

In [19]:
df = gpd.read_file(geo_path)
# df.to_csv("geojson_excel.csv")


In [32]:
def multipolygon_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

df['coordinates'] = df['geometry'].apply(multipolygon_to_coordinates)
del df['geometry']
df.head()

,OBJECTID,adm_nm,adm_cd,adm_cd2,gu,dong,coordinates
0,206,서울특별시 마포구 용강동,1114059,1144059000,마포구,용강동,"[[126.9445281888074, 37.54514799989514], [126...."
1,207,서울특별시 마포구 대흥동,1114060,1144060000,마포구,대흥동,"[[126.94681220459611, 37.5568494513366], [126...."
2,208,서울특별시 마포구 염리동,1114061,1144061000,마포구,염리동,"[[126.94968160411595, 37.54381892668303], [126..."
3,209,서울특별시 마포구 신수동,1114063,1144063000,마포구,신수동,"[[126.94073140875314, 37.546613890545075], [12..."
4,210,서울특별시 마포구 서교동,1114066,1144066000,마포구,서교동,"[[126.91353477929526, 37.54948323708437], [126..."


## 각 동별 인구수 데이터가 필요 (동별, 시간대별, 성별)

## 두번째로는 동별, 연령대별 인구도

In [84]:
# 서울시 중심부의 위도, 경도 입니다. 
center = [37.541, 126.986] 
x,y = yp_df['latitude'].mean(),yp_df['longitude'].mean()
# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다. 
m = folium.Map(location=[x,y], zoom_start=13)

# # Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(geo_data=geo_str,
                  data=move_groupby_dong[2], columns=('dong_name','인구'), key_on='feature.properties.dong', fill_color=colormap, legend_name='10 인구수', ).add_to(m)

# 맵 m을 출력합니다. 

m

ValueError: color should be a string, not a <class 'branca.colormap.LinearColormap'>.